In [1]:
import requests
import pandas as pd
import os
from api_key import census_api


In [2]:
# year = 2014
# industry_int = 2
# industry_num = str(industry_int).zfill(2)
# industry_index = 0

In [3]:
# print(industry_num)

In [4]:
combined_data = pd.DataFrame()

In [5]:
industry_list = ['Agriculture', 'Construction', 'Manufacturing', 'Wholesale', 'Retail', 'Transportation', 'Information', 'Finance and Real Estate', 
                 'Professional, scientific, and management, and administrative and waste management services', 'Education and Health', 
                 'Arts, Entertainment, and Food', 'Others, except public administration', 'Public Administration']  

In [6]:
conurl = f'https://api.census.gov/data/2018/acs/acsse?get=NAME,K202403_014E&for=place:*&key={census_api}'

conesponse = requests.get(conurl)

conjson_response = conesponse.json()

con_data = pd.DataFrame(conjson_response[1:], columns=conjson_response[0])

con_data = con_data.rename(columns={'K202403_014E': 'Total Employed'})

con_data = con_data.dropna()

con_data = con_data.loc[(con_data['Total Employed']) != '0']

con_data['Total Employed'] = con_data['Total Employed'].astype(int)

con_data22 = con_data.sort_values('Total Employed', ascending=False)

con_data22['Industry'] = 'Construction'
con_data22['Year'] = '2022'
con_data22 = con_data22[['NAME', 'Industry', 'Year', 'Total Employed']]
con_data22.shape

(2281, 4)

In [7]:
for industry_index in range(len(industry_list)):  # Loop over the index of industry_list
    industry_int = str(industry_index + 2).zfill(2)  # Construct industry_int based on industry_index
    try:
        industry_data = pd.DataFrame()  # Initialize as an empty DataFrame for each industry
        print(f"Fetching data for industry {industry_list[industry_index]}...")
        for year in range(2014, 2023):
            print(f"Fetching data for year {year}...")
            url = f'https://api.census.gov/data/{year}/acs/acsse?get=NAME,K202403_0{industry_int}E&for=place:*&key={census_api}'
            response = requests.get(url)
            if response.status_code == 200:
                json_response = response.json()
                data = pd.DataFrame(json_response[1:], columns=json_response[0])
                data = data.rename(columns={f'K202403_0{industry_int}E': 'Total Employed'})
                data = data.dropna()
                data = data.loc[data['Total Employed'] != '0']
                data['Industry'] = industry_list[industry_index]
                data['Total Employed'] = data['Total Employed'].astype(int)
                data = data.sort_values('Total Employed', ascending=False)
                data['Year'] = str(year)
                industry_data = pd.concat([industry_data, data], ignore_index=True)  # Concatenate data for each year
                print(f"Data for year {year} fetched successfully.")
                print(f"Size of industry data: {industry_data.shape}")
                print(f"Size of data after concatenation: {data.shape}")
            else:
                print(f"Error fetching data for year {year}: Status code {response.status_code}")
        combined_data = pd.concat([combined_data, industry_data], ignore_index=True)  # Concatenate data for each industry
        print(f"Size of combined data: {combined_data.shape}")
    except Exception as e:
        print(f'Error fetching data for industry {industry_list[industry_index]}: {e}')
print(combined_data)


Fetching data for industry Agriculture...
Fetching data for year 2014...
Data for year 2014 fetched successfully.
Size of industry data: (1581, 6)
Size of data after concatenation: (1581, 6)
Fetching data for year 2015...
Data for year 2015 fetched successfully.
Size of industry data: (3113, 6)
Size of data after concatenation: (1532, 6)
Fetching data for year 2016...
Data for year 2016 fetched successfully.
Size of industry data: (4628, 6)
Size of data after concatenation: (1515, 6)
Fetching data for year 2017...
Data for year 2017 fetched successfully.
Size of industry data: (6168, 6)
Size of data after concatenation: (1540, 6)
Fetching data for year 2018...
Data for year 2018 fetched successfully.
Size of industry data: (7749, 6)
Size of data after concatenation: (1581, 6)
Fetching data for year 2019...
Data for year 2019 fetched successfully.
Size of industry data: (9273, 6)
Size of data after concatenation: (1524, 6)
Fetching data for year 2020...
Error fetching data for year 2020

In [8]:
combined_data


,NAME,Total Employed,state,place,Industry,Year
0,"Houston city, Texas",39821,48,35000,Agriculture,2014
1,"Salinas city, California",16668,06,64224,Agriculture,2014
2,"Bakersfield city, California",16167,06,03526,Agriculture,2014
3,"Midland city, Texas",14643,48,48072,Agriculture,2014
4,"Oxnard city, California",12251,06,54652,Agriculture,2014
...,...,...,...,...,...,...
228611,"Ypsilanti city, Michigan",42,26,89140,Public Administration,2022
228612,"Crystal city, Minnesota",40,27,14158,Public Administration,2022
228613,"Wade Hampton CDP, South Carolina",39,45,73870,Public Administration,2022
228614,"Soledad city, California",38,06,72520,Public Administration,2022


In [9]:
combined_income_data = pd.DataFrame()

In [10]:
income_industry_list = ['Agriculture, forestry, fishing and hunting, and mining', 'Agriculture, forestry, fishing and hunting, and mining:!!Agriculture, forestry, fishing and hunting',
                        'Agriculture, forestry, fishing and hunting, and mining:!!Mining, quarrying, and oil and gas extraction', 'Construction', 'Manufacturing', 'Wholesale trade',
                        'Retail trade', 'Transportation and warehousing, and utilities', 'Transportation and warehousing, and utilities:!!Transportation and warehousing',
                        'Transportation and warehousing, and utilities:!!Utilities', 'Information', 'Finance and insurance, and real estate, and rental and leasing',
                        'Finance and insurance, and real estate, and rental and leasing:!!Finance and insurance', 'Finance and insurance, and real estate, and rental and leasing:!!Real estate and rental and leasing',
                        'Professional, scientific, and management, and administrative, and waste management services', 'Professional, scientific, and management, and administrative, and waste management services:!!Professional, scientific, and technical services',
                        'Professional, scientific, and management, and administrative, and waste management services:!!Management of companies and enterprises',
                        'Professional, scientific, and management, and administrative, and waste management services:!!Administrative and support and waste management services',
                        'Educational services, and health care and social assistance', 'Educational services, and health care and social assistance:!!Educational services',
                        'Educational services, and health care and social assistance:!!Health care and social assistance', 'Arts, entertainment, and recreation, and accommodation and food services',
                        'Arts, entertainment, and recreation, and accommodation and food services:!!Arts, entertainment, and recreation', 'Arts, entertainment, and recreation, and accommodation and food services:!!Accommodation and food services',
                        'Other services, except public administration', 'Public administration']

In [11]:
for industry_index in range(len(income_industry_list)):  # Loop over the index of industry_list
    industry_int = str(industry_index + 2).zfill(2)  # Construct industry_int based on industry_index
    try:
        salary_data = pd.DataFrame()  # Initialize as an empty DataFrame for each industry
        print(f"Fetching data for industry {income_industry_list[industry_index]}...")
        for year in range(2014, 2023):
            print(f"Fetching data for year {year}...")
            url = f'https://api.census.gov/data/{year}/acs/acs1?get=NAME,B24031_0{industry_int}E&for=place:*&key={census_api}'
            response = requests.get(url)
            if response.status_code == 200:
                json_response = response.json()
                data = pd.DataFrame(json_response[1:], columns=json_response[0])
                data = data.rename(columns={f'B24031_0{industry_int}E': 'Income'})
                data = data.dropna()
                data = data.loc[data['Income'] != '0']
                data['Industry'] = income_industry_list[industry_index]
                data['Income'] = data['Income'].astype(int)
                data = data.sort_values('Income', ascending=False)
                data['Year'] = str(year)
                salary_data = pd.concat([salary_data, data], ignore_index=True)  # Concatenate data for each year
                print(f"Data for year {year} fetched successfully.")
                print(f"Size of industry data: {salary_data.shape}")
                print(f"Size of data after concatenation: {data.shape}")
            else:
                print(f"Error fetching data for year {year}: Status code {response.status_code}")
        combined_income_data = pd.concat([combined_income_data, salary_data], ignore_index=True)  # Concatenate data for each industry
        print(f"Size of combined data: {combined_income_data.shape}")
    except Exception as e:
        print(f'Error fetching data for industry {income_industry_list[industry_index]}: {e}')
print(combined_income_data)


Fetching data for industry Agriculture, forestry, fishing and hunting, and mining...
Fetching data for year 2014...
Data for year 2014 fetched successfully.
Size of industry data: (590, 6)
Size of data after concatenation: (590, 6)
Fetching data for year 2015...
Data for year 2015 fetched successfully.
Size of industry data: (1186, 6)
Size of data after concatenation: (596, 6)
Fetching data for year 2016...
Data for year 2016 fetched successfully.
Size of industry data: (1790, 6)
Size of data after concatenation: (604, 6)
Fetching data for year 2017...
Data for year 2017 fetched successfully.
Size of industry data: (2404, 6)
Size of data after concatenation: (614, 6)
Fetching data for year 2018...
Data for year 2018 fetched successfully.
Size of industry data: (3034, 6)
Size of data after concatenation: (630, 6)
Fetching data for year 2019...
Data for year 2019 fetched successfully.
Size of industry data: (3667, 6)
Size of data after concatenation: (633, 6)
Fetching data for year 2020.

In [12]:
combined_income_data

,NAME,Income,state,place,Industry,Year
0,"Arlington CDP, Virginia",250001,51,03000,"Agriculture, forestry, fishing and hunting, an...",2014
1,"San Ramon city, California",175919,06,68378,"Agriculture, forestry, fishing and hunting, an...",2014
2,"League City city, Texas",151946,48,41980,"Agriculture, forestry, fishing and hunting, an...",2014
3,"Fall River city, Massachusetts",145450,25,23000,"Agriculture, forestry, fishing and hunting, an...",2014
4,"Peoria city, Arizona",126689,04,54050,"Agriculture, forestry, fishing and hunting, an...",2014
...,...,...,...,...,...,...
128565,"Gary city, Indiana",22339,18,27000,Public administration,2022
128566,"Canton city, Ohio",12340,39,12000,Public administration,2022
128567,"The Villages CDP, Florida",2511,12,71625,Public administration,2022
128568,"Cicero town, Illinois",-666666666,17,14351,Public administration,2022


In [13]:
male_income = pd.DataFrame()

In [14]:
for industry_index in range(len(income_industry_list)):  # Loop over the index of industry_list
    industry_int = str(industry_index + 2).zfill(2)  # Construct industry_int based on industry_index
    try:
        male_salary_data = pd.DataFrame()  # Initialize as an empty DataFrame for each industry
        print(f"Fetching data for males in industry {income_industry_list[industry_index]}...")
        for year in range(2014, 2023):
            print(f"Fetching data for year {year}...")
            url = f'https://api.census.gov/data/{year}/acs/acs1?get=NAME,B24032_0{industry_int}E&for=place:*&key={census_api}'
            response = requests.get(url)
            if response.status_code == 200:
                json_response = response.json()
                data = pd.DataFrame(json_response[1:], columns=json_response[0])
                data = data.rename(columns={f'B24032_0{industry_int}E': 'Income'})
                data = data.dropna()
                data = data.loc[data['Income'] != '0']
                data['Industry'] = income_industry_list[industry_index]
                data['Income'] = data['Income'].astype(int)
                data = data.sort_values('Income', ascending=False)
                data['Year'] = str(year)
                male_salary_data = pd.concat([male_salary_data, data], ignore_index=True)  # Concatenate data for each year
                print(f"Data for year {year} fetched successfully.")
                print(f"Size of industry data: {male_salary_data.shape}")
                print(f"Size of data after concatenation: {data.shape}")
            else:
                print(f"Error fetching data for year {year}: Status code {response.status_code}")
        male_income = pd.concat([male_income, male_salary_data], ignore_index=True)  # Concatenate data for each industry
        print(f"Size of combined data: {male_income.shape}")
    except Exception as e:
        print(f'Error fetching data for males in industry {income_industry_list[industry_index]}: {e}')
print(male_income)


Fetching data for males in industry Agriculture, forestry, fishing and hunting, and mining...
Fetching data for year 2014...
Data for year 2014 fetched successfully.
Size of industry data: (591, 6)
Size of data after concatenation: (591, 6)
Fetching data for year 2015...
Data for year 2015 fetched successfully.
Size of industry data: (1187, 6)
Size of data after concatenation: (596, 6)
Fetching data for year 2016...
Data for year 2016 fetched successfully.
Size of industry data: (1792, 6)
Size of data after concatenation: (605, 6)
Fetching data for year 2017...
Data for year 2017 fetched successfully.
Size of industry data: (2406, 6)
Size of data after concatenation: (614, 6)
Fetching data for year 2018...
Data for year 2018 fetched successfully.
Size of industry data: (3035, 6)
Size of data after concatenation: (629, 6)
Fetching data for year 2019...
Data for year 2019 fetched successfully.
Size of industry data: (3668, 6)
Size of data after concatenation: (633, 6)
Fetching data for y

In [15]:
male_income

,NAME,Income,state,place,Industry,Year
0,"Missouri City city, Texas",156105,48,48804,"Agriculture, forestry, fishing and hunting, an...",2014
1,"League City city, Texas",152357,48,41980,"Agriculture, forestry, fishing and hunting, an...",2014
2,"Tracy city, California",150026,06,80238,"Agriculture, forestry, fishing and hunting, an...",2014
3,"Fall River city, Massachusetts",145450,25,23000,"Agriculture, forestry, fishing and hunting, an...",2014
4,"Peoria city, Arizona",126689,04,54050,"Agriculture, forestry, fishing and hunting, an...",2014
...,...,...,...,...,...,...
128591,"Pine Hills CDP, Florida",-666666666,12,56825,Public administration,2022
128592,"Gary city, Indiana",-666666666,18,27000,Public administration,2022
128593,"Kissimmee city, Florida",-666666666,12,36950,Public administration,2022
128594,"Doral city, Florida",-666666666,12,17935,Public administration,2022


In [16]:
female_income = pd.DataFrame()

In [17]:
for industry_index in range(len(income_industry_list)):  # Loop over the index of industry_list
    industry_int = str(industry_index + 29).zfill(2)  # Construct industry_int based on industry_index
    try:
        female_salary_data = pd.DataFrame()  # Initialize as an empty DataFrame for each industry
        print(f"Fetching data for females in industry {income_industry_list[industry_index]}...")
        for year in range(2014, 2023):
            print(f"Fetching data for year {year}...")
            url = f'https://api.census.gov/data/{year}/acs/acs1?get=NAME,B24032_0{industry_int}E&for=place:*&key={census_api}'
            response = requests.get(url)
            if response.status_code == 200:
                json_response = response.json()
                data = pd.DataFrame(json_response[1:], columns=json_response[0])
                data = data.rename(columns={f'B24032_0{industry_int}E': 'Income'})
                data = data.dropna()
                data = data.loc[data['Income'] != '0']
                data['Industry'] = income_industry_list[industry_index]
                data['Income'] = data['Income'].astype(int)
                data = data.sort_values('Income', ascending=False)
                data['Year'] = str(year)
                female_salary_data = pd.concat([female_salary_data, data], ignore_index=True)  # Concatenate data for each year
                print(f"Data for year {year} fetched successfully.")
                print(f"Size of industry data: {female_salary_data.shape}")
                print(f"Size of data after concatenation: {data.shape}")
            else:
                print(f"Error fetching data for year {year}: Status code {response.status_code}")
        female_income = pd.concat([female_income, female_salary_data], ignore_index=True)  # Concatenate data for each industry
        print(f"Size of combined data: {female_income.shape}")
    except Exception as e:
        print(f'Error fetching data for females in industry {income_industry_list[industry_index]}: {e}')
print(female_income)


Fetching data for females in industry Agriculture, forestry, fishing and hunting, and mining...
Fetching data for year 2014...
Data for year 2014 fetched successfully.
Size of industry data: (591, 6)
Size of data after concatenation: (591, 6)
Fetching data for year 2015...
Data for year 2015 fetched successfully.
Size of industry data: (1187, 6)
Size of data after concatenation: (596, 6)
Fetching data for year 2016...
Data for year 2016 fetched successfully.
Size of industry data: (1792, 6)
Size of data after concatenation: (605, 6)
Fetching data for year 2017...
Data for year 2017 fetched successfully.
Size of industry data: (2406, 6)
Size of data after concatenation: (614, 6)
Fetching data for year 2018...
Data for year 2018 fetched successfully.
Size of industry data: (3035, 6)
Size of data after concatenation: (629, 6)
Fetching data for year 2019...
Data for year 2019 fetched successfully.
Size of industry data: (3668, 6)
Size of data after concatenation: (633, 6)
Fetching data for

In [18]:
female_income

,NAME,Income,state,place,Industry,Year
0,"Arlington CDP, Virginia",250001,51,03000,"Agriculture, forestry, fishing and hunting, an...",2014
1,"San Marcos city, California",230485,06,68196,"Agriculture, forestry, fishing and hunting, an...",2014
2,"San Ramon city, California",175257,06,68378,"Agriculture, forestry, fishing and hunting, an...",2014
3,"Arlington city, Texas",125390,48,04000,"Agriculture, forestry, fishing and hunting, an...",2014
4,"Sugar Land city, Texas",122394,48,70808,"Agriculture, forestry, fishing and hunting, an...",2014
...,...,...,...,...,...,...
128591,"Pleasanton city, California",-666666666,06,57792,Public administration,2022
128592,"Alpharetta city, Georgia",-666666666,13,01696,Public administration,2022
128593,"The Villages CDP, Florida",-666666666,12,71625,Public administration,2022
128594,"Port Charlotte CDP, Florida",-666666666,12,58350,Public administration,2022


In [19]:
male_income['Sex'] = 'Male'
female_income['Sex'] = 'Female'
male_income = male_income.loc[(male_income['Income'] > 0)]
female_income = female_income.loc[(female_income['Income'] > 0)]

In [20]:
full_data = pd.concat([male_income, female_income])
full_data.reset_index(drop=True, inplace=True)

In [21]:
full_data

,NAME,Income,state,place,Industry,Year,Sex
0,"Missouri City city, Texas",156105,48,48804,"Agriculture, forestry, fishing and hunting, an...",2014,Male
1,"League City city, Texas",152357,48,41980,"Agriculture, forestry, fishing and hunting, an...",2014,Male
2,"Tracy city, California",150026,06,80238,"Agriculture, forestry, fishing and hunting, an...",2014,Male
3,"Fall River city, Massachusetts",145450,25,23000,"Agriculture, forestry, fishing and hunting, an...",2014,Male
4,"Peoria city, Arizona",126689,04,54050,"Agriculture, forestry, fishing and hunting, an...",2014,Male
...,...,...,...,...,...,...,...
211382,"Ankeny city, Iowa",18661,19,02305,Public administration,2022,Female
211383,"Provo city, Utah",11111,49,62470,Public administration,2022,Female
211384,"South Gate city, California",9681,06,73080,Public administration,2022,Female
211385,"Westland city, Michigan",4385,26,86000,Public administration,2022,Female
